# Corpus exploration

Can we find evidence among all contexts of positive examples of a property by comparing them to the contexts of all negative examples?

Here, similar concepts should come in handy, as there should be a lot of overlap in their contexts. Distinctive aspects should be all the more salient. 



In [1]:
import pandas as pd
import os

In [ ]:
# run extract_contexts.py (vm)
# dowload contexts - out in ../contexts/
# run get_tfidf.py
# run process_tfidf.py
# annotate evidence candidates
# run 

In [2]:
# sanity check giga

path_giga = '../contexts/giga_full/vocab/'
path_wiki =  '../contexts/wiki/vocab/'
path_vocab = '../data/vocab.txt'

with open(path_vocab) as infile:
    vocab = set(infile.read().strip().split('\n'))
print(len(vocab))

giga_vocab = [f.split('.')[0] for f in os.listdir(path_giga)]
print(len(giga_vocab))

wiki_vocab = [f.split('.')[0] for f in os.listdir(path_wiki)]
print(len(wiki_vocab))

1780
1445
1669


# Finding candidates for property-evidece through TF-idf analysis 

In [5]:
# get overview of counts
from process_tfidf import get_table
from analyze_evidence import get_evidence_table, get_properties
import pandas as pd

In [212]:
# before annotation 

prop = 'female'
model = 'giga_full'
cnt_type = 'raw'
max_features = 10000
category = 'all-pos'
pos_dicts, neg_dicts = get_table(model, prop, category, 
                                 cnt_type, max_features, 
                                 rank_by='n_concepts', top_n=20)
df_pos = pd.DataFrame(pos_dicts)
df_neg = pd.DataFrame(neg_dicts)
df_pos

../analysis/giga_full/tfidf_aggregated_concept_scores-raw-10000/female-pos/all-pos/female-pos.csv


,context,n_concepts,mean_tfidf,mean_diff
0,herself,77,0.02,0.02
1,she,72,0.05,0.04
2,beautiful,68,0.02,0.02
3,lady,65,0.02,0.02
4,anna,63,0.02,0.02
5,love,63,0.02,0.02
6,mary,61,0.03,0.03
7,actress,60,0.03,0.03
8,elizabeth,60,0.04,0.03
9,maria,59,0.03,0.03


In [8]:
df_neg

,context,n_concepts,mean_tfidf,mean_diff
0,himself,98,0.01,0.01
1,michael,97,0.01,0.01
2,david,95,0.01,0.01
3,steve,94,0.01,0.01
4,brian,92,0.01,0.01
5,failed,91,0.00,0.00
6,signed,91,0.00,0.00
7,peter,91,0.01,0.01
8,james,90,0.01,0.01
9,bob,89,0.01,0.01


## Consider semantic categories

In [213]:
prop = 'fly'
model = 'giga_full'
cnt_type = 'raw'
max_features = 10000
category = 'bird'
pos_dicts, neg_dicts = get_table(model, prop, category, 
                                 cnt_type, max_features, 
                                 rank_by='n_concepts', top_n=20)
df_pos = pd.DataFrame(pos_dicts)
df_neg = pd.DataFrame(neg_dicts)
df_pos

../analysis/giga_full/tfidf_aggregated_concept_scores-raw-10000/fly-pos/bird/fly-pos.csv


,context,n_concepts,mean_tfidf,mean_diff
0,bird,21,0.04,0.04
1,birds,20,0.03,0.03
2,nest,17,0.04,0.04
3,flew,17,0.01,0.01
4,wings,17,0.01,0.01
5,flying,15,0.03,0.03
6,population,15,0.02,0.01
7,flies,15,0.02,0.02
8,fly,14,0.01,0.01
9,billed,14,0.07,0.07


## Annotation procedure


### File handling:

* Annotate per semantic category and for all positive concepts
* Files: ../analysis/giga_full/annotation-tfidf-top20-raw-10000/dangerous-pos/
* Aggregate all evidence words in one file (see below)
* Copy file cp: annotation.csv annotation-done.csv
* Annotate annotation-done.csv and save


### Annotation framework


| evidence type                                                       | label | comment                     | examples                         |   |   |   |   |
|---------------------------------------------------------------------|-------|-----------------------------|----------------------------------|---|---|---|---|
| word expressing the property                                        | p     |                             | Red-red, lay_eggs-eggs, used_in_cooking-cook                       |   |   |   |   |
| near synonym of property word                                       | n     |                             | Cold-frozen                      |   |   |   |   |
| related to property                                                 | r     |                             | Swim-sea                         |   |   |   |   |
| instance of concept having the property                             | i     |                             | Juicy-pineapple                  |   |   |   |   |
| culturally associated with property, but no logical relation (bias) | b     | possibly covered by related | Dangerous-man, dangerous-soldier, female-beautiful |   |   |   |   |
| logical implication                                                 | l     |                             | Female-pregnant                  |
| unrelated                                                           | u     |                             |                                  |   |   |   |   |


change order of logical and bias

## Inspect annotations

### Check annotation status

In [2]:
from analyze_evidence import show_annotation_status

In [6]:
model_name = 'wiki'
show_annotation_status(model_name)

completed:

black-pos
blue-pos
cold-pos
dangerous-pos
female-pos
fly-pos
green-pos
hot-pos
juicy-pos
lay_eggs-pos
made_of_wood-pos
red-pos
roll-pos
round-pos
square-pos
sweet-pos
swim-pos
used_in_cooking-pos
warm-pos

Incomplete:

wheels-pos
wings-pos
yellow-pos


In [7]:
# prepare new file:
from analyze_evidence import summarize_annotation_table

#set up new annotation

prop = 'wheels'
model_name = 'wiki'
summarize_annotation_table(prop, model_name)

### Check annotation consistency


In [8]:
from analyze_evidence import get_property_annotations
import pandas as pd
pd.set_option('max_rows', None)

In [9]:
# get overview

prop = 'used_in_cooking'
model_name = 'wiki'
df = get_property_annotations(model_name, prop)  
df = df.loc[df['evidence'] == True]
df

,object,all-pos,fruit,food,measure,plant,vegetable,cutlery,tool,animal,evidence
sweet,r,r,r,r,-,r,r,-,-,-,True
eaten,r,r,r,r,r,r,r,r,-,r,True
potato,i,i,i,i,-,i,i,-,-,-,True
consumed,-,-,r,r,-,-,-,-,-,-,True
seeds,-,-,r,-,-,r,r,-,-,-,True
mixture,r,r,r,r,r,r,r,-,-,-,True
seed,-,-,r,-,-,-,-,-,-,-,True
made,-,-,r,-,-,-,-,-,-,-,True
rice,i,i,i,-,i,i,i,-,-,-,True
chicken,i,i,i,i,-,i,-,-,-,-,True


### Compare annotations across corpora to make sure they are consistent

In [55]:
from analyze_evidence import check_consistency_across_corpora, get_property_annotations

In [113]:

prop = 'used_in_cooking'

model_name = 'giga_full'
df_giga = get_property_annotations(model_name, prop) 

model_name = 'wiki'
df_wiki = get_property_annotations(model_name, prop) 

df = check_consistency_across_corpora(df_wiki, df_giga)
df = df.loc[df['in_line'] == False]
df       

,word,giga,wiki,in_line


### Property - concept overview

* avoid duplicate counting

In [10]:
from analyze_evidence import get_concept_evidence_counts, get_evidence_words
import pandas as pd

In [11]:
prop = 'dangerous'
model_name = 'giga_full'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
13,killed,r,0.98,0.68,0.80,0.68,42,62,0.02,1,51
3,shoot,r,0.98,0.66,0.79,0.66,41,62,0.02,1,51
0,shooting,i,1.00,0.61,0.76,0.61,38,62,0.00,0,51
15,lethal,r,1.00,0.61,0.76,0.61,38,62,0.00,0,51
7,killing,i,0.91,0.65,0.75,0.65,40,62,0.08,4,51
10,soldier,b,1.00,0.60,0.75,0.60,37,62,0.00,0,51
1,armed,r,0.97,0.60,0.74,0.60,37,62,0.02,1,51
17,target,r,1.00,0.58,0.73,0.58,36,62,0.00,0,51
21,dead,r,0.95,0.60,0.73,0.60,37,62,0.04,2,51
14,attack,i,0.95,0.60,0.73,0.60,37,62,0.04,2,51


In [12]:
prop = 'dangerous'
model_name = 'wiki'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
8,killing,i,0.98,0.66,0.79,0.66,47,71,0.02,1,59
5,kills,r,0.94,0.66,0.78,0.66,47,71,0.05,3,59
0,weapon,i,0.98,0.63,0.77,0.63,45,71,0.02,1,59
11,attack,i,0.98,0.63,0.77,0.63,45,71,0.02,1,59
4,gun,i,0.98,0.62,0.76,0.62,44,71,0.02,1,59
12,kill,r,0.96,0.62,0.75,0.62,44,71,0.03,2,59
13,killed,r,0.96,0.61,0.74,0.61,43,71,0.03,2,59
10,dangerous,p,0.95,0.59,0.73,0.59,42,71,0.03,2,59
14,target,r,0.95,0.59,0.73,0.59,42,71,0.03,2,59
1,firing,i,1.00,0.56,0.72,0.56,40,71,0.00,0,59


In [9]:
prop = 'dangerous'
model_name = 'giga_full'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
13,killed,r,0.98,0.68,0.80,0.68,42,62,0.02,1,51
3,shoot,r,0.98,0.66,0.79,0.66,41,62,0.02,1,51
0,shooting,i,1.00,0.61,0.76,0.61,38,62,0.00,0,51
15,lethal,r,1.00,0.61,0.76,0.61,38,62,0.00,0,51
7,killing,i,0.91,0.65,0.75,0.65,40,62,0.08,4,51
10,soldier,b,1.00,0.60,0.75,0.60,37,62,0.00,0,51
1,armed,r,0.97,0.60,0.74,0.60,37,62,0.02,1,51
17,target,r,1.00,0.58,0.73,0.58,36,62,0.00,0,51
21,dead,r,0.95,0.60,0.73,0.60,37,62,0.04,2,51
14,attack,i,0.95,0.60,0.73,0.60,37,62,0.04,2,51


In [26]:
prop = 'female'
model_name = 'giga_full'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
0,herself,i,0.90,0.71,0.79,0.71,77,109,0.06,9,144
1,she,i,0.88,0.66,0.75,0.66,72,109,0.07,10,144
2,beautiful,b,0.88,0.62,0.73,0.62,68,109,0.06,9,144
3,lady,i,0.92,0.60,0.72,0.60,65,109,0.04,6,144
4,anna,i,0.94,0.58,0.72,0.58,63,109,0.03,4,144
6,mary,i,0.94,0.56,0.70,0.56,61,109,0.03,4,144
9,maria,i,0.98,0.54,0.70,0.54,59,109,0.01,1,144
8,elizabeth,i,0.95,0.55,0.70,0.55,60,109,0.02,3,144
10,lovely,b,0.92,0.54,0.68,0.54,59,109,0.03,5,144
7,actress,i,0.90,0.55,0.68,0.55,60,109,0.05,7,144


In [5]:
prop = 'female'
model_name = 'wiki'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
0,she,i,0.91,0.80,0.85,0.80,98,122,0.07,10,150
1,herself,i,0.96,0.72,0.82,0.72,88,122,0.03,4,150
3,beautiful,b,0.94,0.69,0.80,0.69,84,122,0.03,5,150
5,lady,i,0.95,0.66,0.78,0.66,81,122,0.03,4,150
2,woman,i,0.87,0.70,0.77,0.70,85,122,0.09,13,150
4,her,i,0.83,0.69,0.75,0.69,84,122,0.11,17,150
6,mistress,i,0.95,0.58,0.72,0.58,71,122,0.03,4,150
7,mary,i,0.95,0.58,0.72,0.58,71,122,0.03,4,150
9,alice,i,0.92,0.57,0.71,0.57,70,122,0.04,6,150
11,marie,i,0.92,0.57,0.70,0.57,69,122,0.04,6,150


In [27]:
prop = 'red'
model_name = 'giga_full'
#evidence_words_total = get_evidence_words(prop, model)
overview_table = pd.DataFrame(get_concept_evidence_counts(prop, model_name))
overview_table.sort_values('f1', ascending=False).round(2)

,word,evidence_type,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg
0,red,p,0.95,0.46,0.62,0.46,35,76,0.03,2,64
3,wine,i,0.95,0.46,0.62,0.46,35,76,0.03,2,64
8,color,i,0.85,0.46,0.60,0.46,35,76,0.09,6,64
2,plum,i,0.94,0.41,0.57,0.41,31,76,0.03,2,64
1,cherry,i,0.91,0.38,0.54,0.38,29,76,0.05,3,64
21,roses,i,0.85,0.38,0.53,0.38,29,76,0.08,5,64
11,cocktail,i,0.71,0.38,0.50,0.38,29,76,0.19,12,64
6,punch,i,0.72,0.37,0.49,0.37,28,76,0.17,11,64
20,plums,i,0.75,0.36,0.48,0.36,27,76,0.14,9,64
7,peach,i,0.86,0.33,0.48,0.33,25,76,0.06,4,64


## Property - evidence type overview


In [28]:
from analyze_evidence import get_concept_context_overview
import pandas as pd

In [29]:
prop = 'used_in_cooking'
model_name = 'giga_full'
table = get_concept_context_overview(prop, model_name)
df = pd.DataFrame(table).round(2)
df

,evidence,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,total_evidence_words
0,all,0.75,1.00,0.86,1.00,97,97,0.57,32,56,67
1,p,0.99,0.88,0.93,0.88,85,97,0.02,1,56,4
2,n,0.00,0.00,0.00,0.00,0,97,0.00,0,56,0
3,i,0.90,1.00,0.95,1.00,97,97,0.20,11,56,21
4,r,0.78,1.00,0.88,1.00,97,97,0.48,27,56,42
5,b,0.00,0.00,0.00,0.00,0,97,0.00,0,56,0
6,l,0.00,0.00,0.00,0.00,0,97,0.00,0,56,0


In [30]:
prop = 'female'
model_name = 'giga_full'
table = get_concept_context_overview(prop, model_name)
df = pd.DataFrame(table).round(2)
df

,evidence,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,total_evidence_words
0,all,0.51,1.00,0.67,1.00,109,109,0.73,105,144,42
1,p,0.55,0.30,0.39,0.30,33,109,0.19,27,144,1
2,n,0.00,0.00,0.00,0.00,0,109,0.00,0,144,0
3,i,0.64,0.96,0.77,0.96,105,109,0.42,60,144,24
4,r,0.45,0.54,0.49,0.54,59,109,0.49,71,144,5
5,b,0.68,0.90,0.77,0.90,98,109,0.32,46,144,10
6,l,0.80,0.49,0.61,0.49,53,109,0.09,13,144,2


In [4]:
prop = 'dangerous'
model_name = 'giga_full'
table = get_concept_context_overview(prop, model_name)
df = pd.DataFrame(table).round(2)
df

,evidence,p_dist,r_dist,f1_dist,n_concepts_with_ev_pos,p_concepts_with_ev_pos,n_concepts_pos,n_concepts_with_ev_neg,p_concepts_with_ev_neg,n_concepts_neg,total_evidence_words
0,all,0.60,1.00,0.75,62,1.00,62,42,0.82,51,48
1,p,0.94,0.53,0.68,33,0.53,62,2,0.04,51,1
2,n,0.00,0.00,0.00,0,0.00,62,0,0.00,51,0
3,i,0.67,0.94,0.78,58,0.94,62,28,0.55,51,15
4,r,0.64,1.00,0.78,62,1.00,62,35,0.69,51,28
5,b,0.83,0.84,0.83,52,0.84,62,11,0.22,51,4
6,l,0.00,0.00,0.00,0,0.00,62,0,0.00,51,0


## Property overivew

* top distinctiveness
* mean distinctiveness
* n different evidence words
* n evidence words by type

In [76]:
from analyze_evidence import get_prop_overview, get_properties
import pandas as pd
pd.set_option('max_rows', None)

In [166]:
model_name = 'giga_full'
properties = get_properties()
prop_table = []
for prop in properties:
    prop_dict = get_prop_overview(model_name, prop)
    prop_table.append(prop_dict)
df_prop_overview = pd.DataFrame(prop_table)

In [167]:
df_prop_overview.sort_values('max_dist_f1', ascending=False).round(2)

,property,n_evidence_words,p,n,i,r,b,l,combined_p,combined_r,combined_f1,max_dist_p,max_dist_r,max_dist_f1,max_dist_ev,max_dist_t
9,used_in_cooking,67,4,1,21,41,0,0,0.75,1.00,0.86,1.00,0.94,0.97,add,r
0,square,11,1,0,10,0,0,0,0.84,1.00,0.91,1.00,0.73,0.85,rooms,i
19,wheels,20,3,0,4,12,0,1,0.89,0.99,0.94,0.98,0.74,0.84,wheel,p
7,sweet,32,2,0,24,5,1,0,0.69,0.99,0.81,0.98,0.71,0.82,sweet,p
17,cold,9,1,2,4,2,0,0,0.90,1.00,0.95,1.00,0.70,0.82,frozen,n
10,juicy,32,3,0,27,2,0,0,0.71,1.00,0.83,1.00,0.69,0.81,pineapple,i
8,hot,30,2,3,10,15,0,0,0.72,1.00,0.83,0.98,0.68,0.80,heat,p
4,fly,25,7,0,9,9,0,0,0.43,0.95,0.59,0.93,0.70,0.80,flew,p
5,dangerous,48,1,0,15,28,4,0,0.60,1.00,0.75,0.98,0.68,0.80,killed,r
21,swim,27,3,1,10,13,0,0,0.78,0.96,0.86,0.98,0.67,0.80,fish,i


In [170]:
prop_max_f1_dict = dict([(row['property'], row['max_dist_f1']) for i, row in df_prop_overview.iterrows()])

In [172]:
prop_max_f1_dict['female']

0.7897435897435897

# Explore abstraction over evidence words

* a model should be able to generalize over evidence words with similar representations
* cosequently, words such as 'she' and 'her' should have similar effects
* if either of the words occurs in the context of a target,  it should have a similar effect on the target representation

### Exploration idea:

* find closest evidence word neighbor for each evidence word
* For each pair, calculate p, r, f1, etc

### Ideas for further exploration:


**Combination of evidence and distinctiveness of evidence**

Is it possible to find a combination of words that is maximally distinctive between pos and neg class?

Start: use pairs

**Cosine similarity of evidence words - how coherent is the group of evidence words?**

In [14]:
# cosine similarity additions

from gensim.models import KeyedVectors
from analyze_evidence import get_evidence_words
from analyze_evidence import  get_evidence_counts, get_evidence_concept_counts
import pandas as pd
pd.set_option('max_rows', None)

In [15]:
model_path = '../../../../Data/dsm/corpus_exploration/giga_full/sgns_pinit1/sgns_rand_pinit1.words'
model = KeyedVectors.load_word2vec_format(model_path, binary=False)
model_dict = dict()
model_dict['model'] = model
model_dict['name'] = 'giga_full'

In [16]:
#model['drink']

array([-1.60952e-01,  1.57530e-02, -1.67969e-01,  8.74580e-02,
       -1.95405e-01,  1.91984e-01,  4.63920e-02, -9.32600e-03,
        1.99320e-02, -4.14120e-02,  4.94492e-01, -7.88960e-02,
        1.08162e-01, -1.84251e-01,  2.90294e-01,  4.10294e-01,
       -1.52952e-01,  1.07827e-01, -3.72910e-01,  2.52075e-01,
        4.06760e-02, -1.71984e-01, -1.35931e-01, -1.75927e-01,
        1.67236e-01,  4.00370e-02, -5.27400e-03,  1.52229e-01,
        3.35850e-02,  3.89470e-02, -8.23550e-02, -1.60713e-01,
        4.97221e-01, -2.96631e-01, -3.15051e-01,  3.25190e-02,
       -1.63634e-01,  1.07247e-01,  3.84639e-01, -7.01270e-02,
       -1.59091e-01,  1.19302e-01,  4.58700e-02,  2.25011e-01,
       -3.18405e-01,  3.67062e-01, -1.21791e-01,  2.37677e-01,
        8.14080e-02,  6.22110e-02,  3.10542e-01, -5.06000e-02,
        1.42430e-01, -2.09664e-01, -2.14500e-03, -1.71074e-01,
       -1.06285e-01, -1.07251e-01,  2.37800e-02,  3.77904e-01,
       -3.56940e-02,  1.88403e-01,  2.95658e-01,  2.790

In [220]:
# get pair distance

def get_pairs(prop, model_name):
    evidence_words = get_evidence_words(prop, model_name)
    evidence_word_pairs = []
    # get all pairs
    for w1 in evidence_words:
        for w2 in evidence_words:
            pair = {w1, w2}
            if len(pair) == 2 and pair not in evidence_word_pairs:
                evidence_word_pairs.append(pair)
    pair_distance_dict = dict()
    for pair in evidence_word_pairs:
        w1, w2 = sorted(pair)
        sim = model.similarity(w1, w2)
        pair_distance_dict[(w1,  w2)] = sim
    return pair_distance_dict


def get_pair_evidence_table(prop, model_name):
    
    pair_distance_dict = get_pairs(prop, model_name)
    mean_prop_sim = sum(pair_distance_dict.values())/len(pair_distance_dict)

    evidence_words_total = get_evidence_words(prop, model_dict['name'])
    ev_cnt_dict = get_evidence_concept_counts(model_dict['name'], prop, evidence_words_total)
    table = []
    for pair, cos in pair_distance_dict.items():
        d = dict()
        d['pair']  = pair
        d['cos'] = cos
        ev_words = pair
        pair_dict = get_evidence_counts(ev_words, evidence_words_total, ev_cnt_dict)
        d.update(pair_dict)
        d['cos-f1'] = cos + pair_dict['f1']
        table.append(d)
    df = pd.DataFrame(table)
    return df

## Properties with selectivity > 0 in diagnostic experiments

In [221]:
prop = 'female'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Max f1 score single word 0.7897435897435897
Number of pairs above max f1 123
Number of words in top pairs: 34
Ratio individual words by number of pairs: 0.2764227642276423
Mean cos top pairs: 0.26715447154471544


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
3,"(anna, herself)",0.30,0.90,0.80,0.84,0.80,87,109,0.07,10,144,1.14
0,"(herself, she)",0.67,0.87,0.82,0.84,0.82,89,109,0.09,13,144,1.51
2,"(herself, lady)",0.36,0.88,0.81,0.84,0.81,88,109,0.08,12,144,1.21
6,"(actress, herself)",0.39,0.88,0.81,0.84,0.81,88,109,0.08,12,144,1.23
87,"(beautiful, maria)",0.20,0.90,0.79,0.84,0.79,86,109,0.07,10,144,1.04
8,"(herself, maria)",0.28,0.90,0.79,0.84,0.79,86,109,0.07,10,144,1.12
10,"(herself, joan)",0.25,0.89,0.79,0.83,0.79,86,109,0.08,11,144,1.09
86,"(beautiful, elizabeth)",0.20,0.89,0.79,0.83,0.79,86,109,0.08,11,144,1.04
40,"(blond, herself)",0.30,0.87,0.80,0.83,0.80,87,109,0.09,13,144,1.13
1,"(beautiful, herself)",0.22,0.85,0.82,0.83,0.82,89,109,0.11,16,144,1.05


In [222]:
prop = 'sweet'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Max f1 score single word 0.8226950354609929
Number of pairs above max f1 242
Number of words in top pairs: 30
Ratio individual words by number of pairs: 0.12396694214876033
Mean cos top pairs: 0.297314049586777


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
155,"(jam, sweet)",0.18,0.99,0.83,0.90,0.83,68,82,0.02,1,63,1.08
64,"(banana, caramel)",0.27,0.99,0.83,0.90,0.83,68,82,0.02,1,63,1.17
74,"(caramel, jam)",0.23,1.00,0.82,0.90,0.82,67,82,0.00,0,63,1.13
148,"(peach, sweet)",0.33,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.23
151,"(plum, sweet)",0.28,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.17
180,"(banana, jam)",0.19,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.08
63,"(caramel, sweet)",0.37,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.27
11,"(candy, plum)",0.24,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.13
15,"(candy, jam)",0.21,0.99,0.82,0.89,0.82,67,82,0.02,1,63,1.11
153,"(strawberry, sweet)",0.26,0.99,0.80,0.89,0.80,66,82,0.02,1,63,1.14


In [206]:
prop = 'wings'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Percentage of pairs above f1 max individual word', len(pairs_above)/len(df))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property wings as a mean similarity of 0.21551673112339095
Max f1 score single word 0.7422680412371134
Number of pairs above max f1 32
Number of words in top pairs: 15
Ratio individual words by number of pairs: 0.46875
Percentage of pairs above f1 max individual word 0.16842105263157894
Mean cos top pairs: 0.22250000000000003


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
2,"(bird, flies)",0.21,0.92,0.75,0.83,0.75,45,60,0.05,4,77,1.04
1,"(bird, wings)",0.22,0.94,0.73,0.82,0.73,44,60,0.04,3,77,1.05
20,"(birds, flies)",0.32,0.90,0.75,0.82,0.75,45,60,0.06,5,77,1.14
21,"(birds, fly)",0.21,0.88,0.75,0.81,0.75,45,60,0.08,6,77,1.02
3,"(bird, fly)",0.17,0.90,0.73,0.81,0.73,44,60,0.06,5,77,0.98
19,"(birds, wings)",0.22,0.90,0.73,0.81,0.73,44,60,0.06,5,77,1.03
39,"(nest, wings)",0.20,0.95,0.68,0.80,0.68,41,60,0.03,2,77,1.00
5,"(bird, flew)",0.12,0.88,0.72,0.79,0.72,43,60,0.08,6,77,0.91
7,"(bird, wing)",0.09,0.89,0.70,0.79,0.70,42,60,0.06,5,77,0.87
6,"(bird, eggs)",0.25,0.91,0.68,0.78,0.68,41,60,0.05,4,77,1.03


In [205]:
prop = 'lay_eggs'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Percentage of pairs above f1 max individual word', len(pairs_above)/len(df))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property lay_eggs as a mean similarity of 0.24377244614075308
Max f1 score single word 0.7407407407407407
Number of pairs above max f1 134
Number of words in top pairs: 26
Ratio individual words by number of pairs: 0.19402985074626866
Percentage of pairs above f1 max individual word 0.3817663817663818
Mean cos top pairs: 0.2375373134328358


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
29,"(bird, fish)",0.34,0.85,0.88,0.87,0.88,29,33,0.09,5,57,1.21
89,"(duck, fishing)",0.18,0.90,0.82,0.86,0.82,27,33,0.05,3,57,1.04
21,"(bass, eggs)",0.15,0.93,0.79,0.85,0.79,26,33,0.04,2,57,1.01
26,"(egg, fish)",0.33,0.96,0.76,0.85,0.76,25,33,0.02,1,57,1.17
0,"(eggs, fish)",0.39,0.96,0.76,0.85,0.76,25,33,0.02,1,57,1.24
3,"(eggs, salmon)",0.30,0.96,0.76,0.85,0.76,25,33,0.02,1,57,1.15
30,"(birds, fish)",0.51,0.87,0.82,0.84,0.82,27,33,0.07,4,57,1.35
63,"(egg, river)",0.10,1.00,0.73,0.84,0.73,24,33,0.00,0,57,0.94
152,"(birds, fishing)",0.24,0.90,0.79,0.84,0.79,26,33,0.05,3,57,1.08
132,"(bird, fishing)",0.20,0.82,0.85,0.84,0.85,28,33,0.11,6,57,1.03


In [204]:
prop = 'used_in_cooking'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Percentage of pairs above f1 max individual word', len(pairs_above)/len(df))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property used_in_cooking as a mean similarity of 0.20922225265528235
Max f1 score single word 0.9680851063829787
Number of pairs above max f1 162
Number of words in top pairs: 52
Ratio individual words by number of pairs: 0.32098765432098764
Percentage of pairs above f1 max individual word 0.07327001356852103
Mean cos top pairs: 0.24370370370370373


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
1253,"(add, fish)",0.13,1.00,0.98,0.99,0.98,95,97,0.00,0,56,1.12
1230,"(add, olive)",0.27,1.00,0.98,0.99,0.98,95,97,0.00,0,56,1.26
1238,"(add, salt)",0.32,1.00,0.98,0.99,0.98,95,97,0.00,0,56,1.31
941,"(add, egg)",0.23,0.99,0.98,0.98,0.98,95,97,0.02,1,56,1.21
327,"(recipe, serve)",0.09,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.08
819,"(fat, recipe)",0.19,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.17
455,"(add, potato)",0.18,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.16
803,"(cheese, recipe)",0.42,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.40
1235,"(add, cheese)",0.26,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.24
488,"(fish, potato)",0.35,1.00,0.97,0.98,0.97,94,97,0.00,0,56,1.33


In [183]:
prop = 'wheels'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property wheels as a mean similarity of 0.26394965923146196
Max f1 score single word 0.8403361344537814
Number of pairs above max f1 135
Number of words in top pairs: 20
Ratio individual words by number of pairs: 0.14814814814814814
Mean cos top pairs: 0.2637777777777778


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
37,"(parked, wheel)",0.32,0.98,0.88,0.93,0.88,60,68,0.04,1,25,1.25
46,"(rolled, wheel)",0.22,0.98,0.88,0.93,0.88,60,68,0.04,1,25,1.15
126,"(bicycle, mini)",0.16,1.00,0.87,0.93,0.87,59,68,0.00,0,25,1.09
1,"(drove, wheel)",0.22,0.98,0.87,0.92,0.87,59,68,0.04,1,25,1.14
56,"(drive, parked)",0.25,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.15
45,"(mini, wheel)",0.24,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.15
49,"(wheel, wheeled)",0.41,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.32
53,"(bike, wheel)",0.35,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.26
38,"(drivers, wheel)",0.38,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.28
42,"(bicycle, wheel)",0.31,0.98,0.84,0.90,0.84,57,68,0.04,1,25,1.21


In [184]:
prop = 'swim'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property swim as a mean similarity of 0.2685973503947937
Max f1 score single word 0.7969924812030076
Number of pairs above max f1 28
Number of words in top pairs: 24
Ratio individual words by number of pairs: 0.8571428571428571
Mean cos top pairs: 0.38642857142857145


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
0,"(fish, fishing)",0.53,0.98,0.75,0.85,0.75,59,79,0.03,1,38,1.37
8,"(fish, swim)",0.31,0.95,0.76,0.85,0.76,60,79,0.08,3,38,1.15
1,"(fish, waters)",0.32,0.98,0.73,0.84,0.73,58,79,0.03,1,38,1.16
2,"(fish, trout)",0.65,0.98,0.72,0.83,0.72,57,79,0.03,1,38,1.48
3,"(catfish, fish)",0.58,0.98,0.72,0.83,0.72,57,79,0.03,1,38,1.41
17,"(crab, fish)",0.58,0.98,0.72,0.83,0.72,57,79,0.03,1,38,1.41
7,"(fish, swimming)",0.23,0.95,0.73,0.83,0.73,58,79,0.08,3,38,1.06
10,"(duck, fish)",0.29,0.88,0.77,0.82,0.77,61,79,0.21,8,38,1.12
19,"(fish, shrimp)",0.63,0.98,0.71,0.82,0.71,56,79,0.03,1,38,1.45
24,"(fish, sea)",0.30,0.94,0.73,0.82,0.73,58,79,0.11,4,38,1.13


In [186]:
prop = 'square'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property square as a mean similarity of 0.21901492862538857
Max f1 score single word 0.8450704225352113
Number of pairs above max f1 49
Number of words in top pairs: 11
Ratio individual words by number of pairs: 0.22448979591836735
Mean cos top pairs: 0.22795918367346937


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
10,"(desk, rooms)",0.27,1.00,0.88,0.94,0.88,36,41,0.00,0,21,1.21
36,"(phone, sheets)",0.13,1.00,0.85,0.92,0.85,35,41,0.00,0,21,1.05
30,"(phone, photograph)",0.15,0.95,0.88,0.91,0.88,36,41,0.10,2,21,1.06
29,"(photograph, table)",0.11,0.95,0.88,0.91,0.88,36,41,0.10,2,21,1.02
40,"(note, table)",0.14,0.95,0.88,0.91,0.88,36,41,0.10,2,21,1.05
37,"(boxes, sheets)",0.30,0.97,0.85,0.91,0.85,35,41,0.05,1,21,1.21
46,"(boxes, table)",0.18,0.97,0.85,0.91,0.85,35,41,0.05,1,21,1.09
50,"(phone, tables)",0.09,1.00,0.83,0.91,0.83,34,41,0.00,0,21,1.00
45,"(phone, table)",0.11,1.00,0.83,0.91,0.83,34,41,0.00,0,21,1.02
15,"(phone, rooms)",0.16,1.00,0.83,0.91,0.83,34,41,0.00,0,21,1.07


In [187]:
prop = 'dangerous'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property dangerous as a mean similarity of 0.15979651386287322
Max f1 score single word 0.8
Number of pairs above max f1 360
Number of words in top pairs: 44
Ratio individual words by number of pairs: 0.12222222222222222
Mean cos top pairs: 0.18875


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
429,"(lethal, soldier)",0.19,1.00,0.85,0.92,0.85,53,62,0.00,0,51,1.11
149,"(lethal, shoot)",0.19,0.98,0.85,0.91,0.85,53,62,0.02,1,51,1.10
14,"(lethal, shooting)",0.22,1.00,0.84,0.91,0.84,52,62,0.00,0,51,1.13
605,"(dead, lethal)",0.23,0.96,0.85,0.91,0.85,53,62,0.04,2,51,1.14
147,"(killed, shoot)",0.21,0.98,0.84,0.90,0.84,52,62,0.02,1,51,1.11
234,"(firing, lethal)",0.24,1.00,0.82,0.90,0.82,51,62,0.00,0,51,1.14
567,"(attack, lethal)",0.28,0.96,0.84,0.90,0.84,52,62,0.04,2,51,1.17
534,"(killed, lethal)",0.28,0.98,0.82,0.89,0.82,51,62,0.02,1,51,1.18
600,"(lethal, man)",0.19,0.93,0.85,0.89,0.85,53,62,0.08,4,51,1.08
603,"(captured, lethal)",0.10,0.95,0.84,0.89,0.84,52,62,0.06,3,51,0.99


In [188]:
prop = 'hot'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property hot as a mean similarity of 0.2114648007059834
Max f1 score single word 0.803030303030303
Number of pairs above max f1 285
Number of words in top pairs: 30
Ratio individual words by number of pairs: 0.10526315789473684
Mean cos top pairs: 0.18301754385964925


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
85,"(burnt, heat)",0.13,0.96,0.90,0.93,0.90,70,78,0.07,3,43,1.06
58,"(flame, hot)",0.23,0.97,0.88,0.93,0.88,69,78,0.05,2,43,1.16
301,"(burning, steamed)",0.11,0.93,0.91,0.92,0.91,71,78,0.12,5,43,1.04
136,"(crisp, heat)",0.26,0.95,0.90,0.92,0.90,70,78,0.09,4,43,1.18
33,"(cooked, smoke)",0.18,0.96,0.88,0.92,0.88,69,78,0.07,3,43,1.10
165,"(burning, cooked)",0.19,0.96,0.88,0.92,0.88,69,78,0.07,3,43,1.11
113,"(hot, steam)",0.31,0.97,0.86,0.91,0.86,67,78,0.05,2,43,1.22
84,"(burnt, hot)",0.19,0.94,0.87,0.91,0.87,68,78,0.09,4,43,1.10
118,"(burning, hot)",0.22,0.96,0.86,0.91,0.86,67,78,0.07,3,43,1.12
307,"(baked, burning)",0.17,0.96,0.86,0.91,0.86,67,78,0.07,3,43,1.07


In [189]:
prop = 'juicy'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property juicy as a mean similarity of 0.3501005388285604
Max f1 score single word 0.8142857142857143
Number of pairs above max f1 211
Number of words in top pairs: 31
Ratio individual words by number of pairs: 0.14691943127962084
Mean cos top pairs: 0.3416587677725119


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
196,"(fruits, pineapple)",0.41,0.99,0.81,0.89,0.81,67,83,0.02,1,60,1.29
120,"(fruit, pineapple)",0.52,0.96,0.82,0.88,0.82,68,83,0.05,3,60,1.41
222,"(cherry, fruits)",0.25,0.96,0.82,0.88,0.82,68,83,0.05,3,60,1.13
199,"(cherry, pineapple)",0.41,0.97,0.81,0.88,0.81,67,83,0.03,2,60,1.29
204,"(blueberry, pineapple)",0.46,0.99,0.80,0.88,0.80,66,83,0.02,1,60,1.34
381,"(blueberry, papaya)",0.39,0.99,0.80,0.88,0.80,66,83,0.02,1,60,1.27
171,"(pineapple, ripe)",0.31,1.00,0.78,0.88,0.78,65,83,0.00,0,60,1.19
296,"(cherry, papaya)",0.31,0.97,0.80,0.87,0.80,66,83,0.03,2,60,1.19
208,"(berry, pineapple)",0.28,0.98,0.78,0.87,0.78,65,83,0.02,1,60,1.15
452,"(grapefruit, papaya)",0.42,1.00,0.77,0.87,0.77,64,83,0.00,0,60,1.29


## Properties with selectivity < 0 in diagnostic experiments

In [209]:
prop = 'blue'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Percentage of pairs above f1 max individual word', len(pairs_above)/len(df))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property blue as a mean similarity of 0.25039180527840343
Max f1 score single word 0.6666666666666667
Number of pairs above max f1 2
Number of words in top pairs: 3
Ratio individual words by number of pairs: 1.5
Percentage of pairs above f1 max individual word 0.07142857142857142
Mean cos top pairs: 0.155


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
8,"(blue, feather)",0.23,0.76,0.63,0.69,0.63,29,46,0.08,9,106,0.92
7,"(birds, blue)",0.08,0.71,0.63,0.67,0.63,29,46,0.11,12,106,0.74
10,"(blue, feathers)",0.20,0.75,0.59,0.66,0.59,27,46,0.08,9,106,0.86
9,"(bird, blue)",0.11,0.64,0.63,0.64,0.63,29,46,0.15,16,106,0.75
12,"(blue, fish)",0.12,0.67,0.61,0.64,0.61,28,46,0.13,14,106,0.75
0,"(blue, ribbon)",0.34,0.55,0.61,0.58,0.61,28,46,0.22,23,106,0.92
11,"(blue, ice)",0.16,0.49,0.65,0.56,0.65,30,46,0.29,31,106,0.72
21,"(feather, fish)",0.26,0.65,0.48,0.55,0.48,22,46,0.11,12,106,0.81
13,"(birds, feather)",0.34,0.74,0.43,0.55,0.43,20,46,0.07,7,106,0.89
17,"(birds, fish)",0.51,0.58,0.46,0.51,0.46,21,46,0.14,15,106,1.02


In [192]:
prop = 'warm'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property warm as a mean similarity of 0.15660775141282515
Max f1 score single word 0.7384615384615384
Number of pairs above max f1 49
Number of words in top pairs: 11
Ratio individual words by number of pairs: 0.22448979591836735
Mean cos top pairs: 0.15673469387755107


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
7,"(heated, wool)",0.10,0.97,0.88,0.92,0.88,72,82,0.06,2,32,1.02
3,"(heated, lined)",0.16,0.98,0.79,0.88,0.79,65,82,0.03,1,32,1.04
16,"(steam, wool)",0.10,0.93,0.82,0.87,0.82,67,82,0.16,5,32,0.97
5,"(heated, hood)",0.11,0.98,0.77,0.86,0.77,63,82,0.03,1,32,0.97
2,"(covered, heated)",0.19,0.92,0.80,0.86,0.80,66,82,0.19,6,32,1.05
12,"(lined, steam)",0.08,0.94,0.78,0.85,0.78,64,82,0.12,4,32,0.93
22,"(hood, warm)",0.08,0.94,0.78,0.85,0.78,64,82,0.12,4,32,0.93
35,"(hood, lined)",0.08,0.97,0.76,0.85,0.76,62,82,0.06,2,32,0.93
20,"(lined, warm)",0.24,0.93,0.78,0.85,0.78,64,82,0.16,5,32,1.09
42,"(room, wool)",0.11,0.94,0.76,0.84,0.76,62,82,0.12,4,32,0.95


In [193]:
prop = 'fly'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property fly as a mean similarity of 0.20076667074734966
Max f1 score single word 0.8
Number of pairs above max f1 39
Number of words in top pairs: 18
Ratio individual words by number of pairs: 0.46153846153846156
Mean cos top pairs: 0.23769230769230773


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
117,"(bird, fly)",0.17,0.89,0.85,0.87,0.85,34,40,0.04,4,90,1.04
114,"(flew, fly)",0.62,0.94,0.80,0.86,0.80,32,40,0.02,2,90,1.48
124,"(fly, nest)",0.17,0.94,0.80,0.86,0.80,32,40,0.02,2,90,1.04
198,"(birds, flew)",0.15,0.87,0.85,0.86,0.85,34,40,0.06,5,90,1.01
197,"(bird, flew)",0.12,0.87,0.85,0.86,0.85,34,40,0.06,5,90,0.98
204,"(flew, nest)",0.18,0.91,0.80,0.85,0.80,32,40,0.03,3,90,1.03
57,"(bird, flying)",0.16,0.85,0.85,0.85,0.85,34,40,0.07,6,90,1.01
200,"(dove, flew)",0.20,0.87,0.82,0.85,0.82,33,40,0.06,5,90,1.05
237,"(bird, overhead)",0.13,0.89,0.80,0.84,0.80,32,40,0.04,4,90,0.97
118,"(birds, fly)",0.21,0.89,0.80,0.84,0.80,32,40,0.04,4,90,1.05


In [194]:
prop = 'round'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property round as a mean similarity of 0.22896552695892752
Max f1 score single word 0.7737226277372262
Number of pairs above max f1 44
Number of words in top pairs: 16
Ratio individual words by number of pairs: 0.36363636363636365
Mean cos top pairs: 0.12295454545454548


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
68,"(circle, seeds)",0.11,1.00,0.86,0.92,0.86,72,84,0.00,0,18,1.03
47,"(circle, pie)",0.19,1.00,0.86,0.92,0.86,72,84,0.00,0,18,1.12
48,"(pie, ring)",0.18,0.97,0.87,0.92,0.87,73,84,0.11,2,18,1.10
102,"(apple, circle)",0.05,0.99,0.83,0.90,0.83,70,84,0.06,1,18,0.95
100,"(circle, fruits)",0.08,0.99,0.82,0.90,0.82,69,84,0.06,1,18,0.97
103,"(circle, jam)",0.17,0.96,0.83,0.89,0.83,70,84,0.17,3,18,1.06
69,"(ring, seeds)",0.08,0.97,0.82,0.89,0.82,69,84,0.11,2,18,0.97
58,"(circle, fruit)",0.05,0.99,0.81,0.89,0.81,68,84,0.06,1,18,0.94
105,"(fruits, ring)",0.06,0.96,0.82,0.88,0.82,69,84,0.17,3,18,0.95
59,"(fruit, ring)",0.07,0.96,0.82,0.88,0.82,69,84,0.17,3,18,0.95


In [195]:
prop = 'cold'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property cold as a mean similarity of 0.2608181567241748
Max f1 score single word 0.8222222222222222
Number of pairs above max f1 17
Number of words in top pairs: 9
Ratio individual words by number of pairs: 0.5294117647058824
Mean cos top pairs: 0.19588235294117648


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
0,"(bottles, frozen)",0.19,1.00,0.83,0.91,0.83,44,53,0.00,0,24,1.09
11,"(frost, frozen)",0.12,1.00,0.83,0.91,0.83,44,53,0.00,0,24,1.03
10,"(frozen, glasses)",0.11,0.98,0.81,0.89,0.81,43,53,0.04,1,24,1.00
8,"(beer, frozen)",0.14,0.98,0.79,0.88,0.79,42,53,0.04,1,24,1.01
12,"(frozen, vanilla)",0.24,1.00,0.77,0.87,0.77,41,53,0.00,0,24,1.12
9,"(cold, frozen)",0.30,0.95,0.79,0.87,0.79,42,53,0.08,2,24,1.17
26,"(frost, glasses)",0.14,0.98,0.77,0.86,0.77,41,53,0.04,1,24,1.00
4,"(bottles, frost)",0.02,1.00,0.75,0.86,0.75,40,53,0.00,0,24,0.88
2,"(bottles, cold)",0.17,0.95,0.77,0.85,0.77,41,53,0.08,2,24,1.02
32,"(cream, frost)",0.21,0.98,0.75,0.85,0.75,40,53,0.04,1,24,1.06


In [196]:
prop = 'made_of_wood'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property made_of_wood as a mean similarity of 0.19239197425882926
Max f1 score single word 0.7234042553191489
Number of pairs above max f1 27
Number of words in top pairs: 10
Ratio individual words by number of pairs: 0.37037037037037035
Mean cos top pairs: 0.24814814814814815


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
35,"(pine, wooden)",0.39,0.96,0.81,0.88,0.81,47,58,0.06,2,33,1.27
3,"(oak, wooden)",0.35,0.96,0.76,0.85,0.76,44,58,0.06,2,33,1.20
37,"(furniture, wooden)",0.35,0.96,0.76,0.85,0.76,44,58,0.06,2,33,1.19
36,"(natural, wooden)",0.10,0.85,0.81,0.83,0.81,47,58,0.24,8,33,0.93
7,"(furniture, oak)",0.26,1.00,0.71,0.83,0.71,41,58,0.00,0,33,1.09
5,"(oak, pine)",0.55,0.98,0.71,0.82,0.71,41,58,0.03,1,33,1.37
9,"(folding, oak)",0.20,0.93,0.69,0.79,0.69,40,58,0.09,3,33,0.99
12,"(tables, wooden)",0.33,0.89,0.71,0.79,0.71,41,58,0.15,5,33,1.12
34,"(wood, wooden)",0.47,0.89,0.71,0.79,0.71,41,58,0.15,5,33,1.26
31,"(bend, furniture)",0.05,0.91,0.69,0.78,0.69,40,58,0.12,4,33,0.83


In [197]:
prop = 'yellow'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property yellow as a mean similarity of 0.34047222489284146
Max f1 score single word 0.6153846153846154
Number of pairs above max f1 9
Number of words in top pairs: 8
Ratio individual words by number of pairs: 0.8888888888888888
Mean cos top pairs: 0.2277777777777778


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
5,"(apple, yellow)",0.14,0.74,0.72,0.73,0.72,29,40,0.14,10,74,0.87
0,"(peach, yellow)",0.28,0.79,0.65,0.71,0.65,26,40,0.09,7,74,0.99
2,"(melon, yellow)",0.22,0.71,0.68,0.69,0.68,27,40,0.15,11,74,0.91
1,"(apples, yellow)",0.20,0.71,0.68,0.69,0.68,27,40,0.15,11,74,0.89
7,"(flowers, yellow)",0.28,0.73,0.60,0.66,0.60,24,40,0.12,9,74,0.94
6,"(apricot, yellow)",0.29,0.71,0.60,0.65,0.60,24,40,0.14,10,74,0.94
3,"(peaches, yellow)",0.19,0.69,0.60,0.64,0.60,24,40,0.15,11,74,0.83
34,"(apple, flowers)",0.12,0.69,0.60,0.64,0.60,24,40,0.15,11,74,0.76
12,"(apple, peach)",0.33,0.78,0.52,0.63,0.52,21,40,0.08,6,74,0.96
20,"(apples, flowers)",0.34,0.66,0.57,0.61,0.57,23,40,0.16,12,74,0.96


In [198]:
prop = 'red'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property red as a mean similarity of 0.2701235002640522
Max f1 score single word 0.6194690265486725
Number of pairs above max f1 65
Number of words in top pairs: 21
Ratio individual words by number of pairs: 0.3230769230769231
Mean cos top pairs: 0.22676923076923086


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
2,"(red, wine)",0.19,0.93,0.68,0.79,0.68,52,76,0.06,4,64,0.98
1,"(plum, red)",0.23,0.93,0.67,0.78,0.67,51,76,0.06,4,64,1.01
6,"(peach, red)",0.25,0.89,0.66,0.76,0.66,50,76,0.09,6,64,1.01
0,"(cherry, red)",0.22,0.91,0.63,0.74,0.63,48,76,0.08,5,64,0.96
77,"(roses, wine)",0.32,0.89,0.63,0.74,0.63,48,76,0.09,6,64,1.05
19,"(plums, red)",0.20,0.83,0.66,0.74,0.66,50,76,0.16,10,64,0.94
3,"(berry, red)",0.20,0.83,0.64,0.73,0.64,49,76,0.16,10,64,0.93
64,"(color, wine)",0.22,0.87,0.62,0.72,0.62,47,76,0.11,7,64,0.94
68,"(flower, wine)",0.23,0.87,0.62,0.72,0.62,47,76,0.11,7,64,0.96
4,"(blackberry, red)",0.15,0.92,0.59,0.72,0.59,45,76,0.06,4,64,0.87


In [199]:
prop = 'green'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property green as a mean similarity of 0.2709340651632932
Max f1 score single word 0.7913669064748202
Number of pairs above max f1 57
Number of words in top pairs: 26
Ratio individual words by number of pairs: 0.45614035087719296
Mean cos top pairs: 0.19280701754385976


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
208,"(green, lawn)",0.23,0.93,0.84,0.88,0.84,69,82,0.07,5,67,1.12
36,"(branches, green)",0.12,0.94,0.83,0.88,0.83,68,82,0.06,4,67,1.01
37,"(branches, mint)",0.17,0.96,0.79,0.87,0.79,65,82,0.04,3,67,1.04
130,"(forests, green)",0.15,0.96,0.79,0.87,0.79,65,82,0.04,3,67,1.02
10,"(green, trees)",0.24,0.94,0.78,0.85,0.78,64,82,0.06,4,67,1.09
242,"(green, mint)",0.22,0.94,0.78,0.85,0.78,64,82,0.06,4,67,1.07
131,"(forests, mint)",0.13,0.95,0.77,0.85,0.77,63,82,0.04,3,67,0.98
209,"(lawn, mint)",0.11,0.93,0.78,0.85,0.78,64,82,0.07,5,67,0.96
225,"(gardens, green)",0.18,0.93,0.78,0.85,0.78,64,82,0.07,5,67,1.03
171,"(green, pine)",0.23,0.93,0.78,0.85,0.78,64,82,0.07,5,67,1.08


In [200]:
prop = 'black'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:10]

Property black as a mean similarity of 0.17409284133464098
Max f1 score single word 0.7142857142857143
Number of pairs above max f1 16
Number of words in top pairs: 9
Ratio individual words by number of pairs: 0.5625
Mean cos top pairs: 0.148125


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
5,"(black, eye)",0.15,0.96,0.78,0.86,0.78,49,63,0.04,2,45,1.01
7,"(black, mouse)",0.12,0.96,0.71,0.82,0.71,45,63,0.04,2,45,0.94
23,"(bull, eye)",0.19,0.98,0.68,0.80,0.68,43,63,0.02,1,45,1.00
2,"(black, bull)",0.13,0.96,0.68,0.80,0.68,43,63,0.04,2,45,0.93
34,"(eye, mouse)",0.18,1.00,0.65,0.79,0.65,41,63,0.00,0,45,0.97
18,"(eye, jackets)",0.06,0.88,0.70,0.78,0.70,44,63,0.13,6,45,0.84
6,"(bird, black)",0.09,0.88,0.70,0.78,0.70,44,63,0.13,6,45,0.87
25,"(bull, mouse)",0.16,0.98,0.63,0.77,0.63,40,63,0.02,1,45,0.93
1,"(black, jackets)",0.19,0.88,0.67,0.76,0.67,42,63,0.13,6,45,0.95
4,"(birds, black)",0.08,0.89,0.65,0.75,0.65,41,63,0.11,5,45,0.83


In [202]:
prop = 'roll'
model_name = model_dict['name']
df = get_pair_evidence_table(prop, model_name).sort_values('f1', ascending=False).round(2)
max_f1_single = prop_max_f1_dict[prop]
total_words = set()
pairs_above = [row['pair'] for i, row in df.iterrows() if row['f1'] > max_f1_single]
[total_words.update(pair) for pair in pairs_above]
cosines_top = [row['cos']  for i, row in df.iterrows() if row['f1'] > max_f1_single]
mean_cos_top = sum(cosines_top)/len(cosines_top)
print('Max f1 score single word', max_f1_single)
print('Number of pairs above max f1', len(pairs_above))
print('Number of words in top pairs:', len(total_words))
print('Ratio individual words by number of pairs:', len(total_words)/len(pairs_above))
print('Mean cos top pairs:', mean_cos_top)
df[:len(pairs_above)]

Property roll as a mean similarity of 0.19517081098800357
Max f1 score single word 0.6753246753246753
Number of pairs above max f1 23
Number of words in top pairs: 10
Ratio individual words by number of pairs: 0.43478260869565216
Mean cos top pairs: 0.23304347826086955


,pair,cos,p,r,f1,p_pos,n_pos,t_pos,p_neg,n_neg,t_neg,cos-f1
43,"(roll, truck)",0.10,0.88,0.70,0.78,0.70,35,50,0.15,5,33,0.88
46,"(rolling, rolls)",0.38,0.88,0.70,0.78,0.70,35,50,0.15,5,33,1.16
52,"(roll, rolls)",0.52,0.87,0.68,0.76,0.68,34,50,0.15,5,33,1.28
40,"(rolling, truck)",0.13,0.89,0.66,0.76,0.66,33,50,0.12,4,33,0.89
47,"(roll, rolling)",0.56,0.91,0.64,0.75,0.64,32,50,0.09,3,33,1.31
38,"(roll, train)",0.04,0.89,0.64,0.74,0.64,32,50,0.12,4,33,0.79
29,"(raced, rolling)",0.18,0.89,0.64,0.74,0.64,32,50,0.12,4,33,0.92
32,"(raced, roll)",0.13,0.89,0.64,0.74,0.64,32,50,0.12,4,33,0.88
42,"(rolls, truck)",0.15,0.85,0.66,0.74,0.66,33,50,0.18,6,33,0.89
16,"(rolls, stops)",0.13,0.81,0.68,0.74,0.68,34,50,0.24,8,33,0.87


# Analysis of property evidence in concept-contexts [to adapt to new setup]

# Property types

In [17]:
from analyze_evidence import get_prop_collection_overview, get_prop_types
import pandas as pd

In [23]:
# create overview table

prop_collection_dict, collection_prop_dict = get_prop_types()
table = []
for collection, props in collection_prop_dict.items():
    props_line = ' '.join(props)
    d = dict()
    d['prop_type'] = collection
    d['properties'] = props_line
    table.append(d)
df = pd.DataFrame(table)
df

,prop_type,properties
0,perceptual-shape,round square
1,percetual-heat,warm cold hot
2,perceptual-color,green red blue yellow black
3,activities,roll swim lay_eggs fly
4,complex,used_in_cooking dangerous
5,parts,wheels wings
6,perceptual,sweet juicy
7,part-material,made_of_wood
8,gender,female


In [24]:
print(df.to_latex(index=False))

\begin{tabular}{ll}
\toprule
        prop\_type &                   properties \\
\midrule
 perceptual-shape &                 round square \\
   percetual-heat &                warm cold hot \\
 perceptual-color &  green red blue yellow black \\
       activities &       roll swim lay\_eggs fly \\
          complex &    used\_in\_cooking dangerous \\
            parts &                 wheels wings \\
       perceptual &                  sweet juicy \\
    part-material &                 made\_of\_wood \\
           gender &                       female \\
\bottomrule
\end{tabular}



In [29]:
props = get_properties()
model = 'giga_full'
collection_table = get_prop_collection_overview(props, model)
df = pd.DataFrame(collection_table).sort_values('collection')
print(df.to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
       collection &  n\_props &  n\_ev &  total\_pos &  p\_ev\_pos &  total\_neg &  p\_ev\_neg \\
\midrule
       activities &        4 &    15 &        200 &      0.45 &        218 &      0.17 \\
          complex &        2 &     6 &        146 &      0.61 &        107 &      0.18 \\
           gender &        1 &     6 &        109 &      0.95 &        144 &      0.33 \\
    part-material &        1 &     2 &         43 &      0.05 &         33 &      0.18 \\
            parts &        2 &     3 &        125 &      0.46 &        102 &      0.06 \\
       perceptual &        2 &    13 &        106 &      0.77 &        123 &      0.23 \\
 perceptual-color &        5 &    10 &        126 &      0.64 &        356 &      0.09 \\
 perceptual-shape &        2 &     1 &         69 &      0.03 &         39 &      0.05 \\
   percetual-heat &        3 &     5 &        132 &      0.46 &         99 &      0.17 \\
\bottomrule
\end{tabular}



In [28]:
props = get_properties()
model = 'wiki'
collection_table = get_prop_collection_overview(props, model)
df = pd.DataFrame(collection_table).sort_values('collection')
print(df.to_latex(index=False))

\begin{tabular}{lrrrrrr}
\toprule
       collection &  n\_props &  n\_ev &  total\_pos &  p\_ev\_pos &  total\_neg &  p\_ev\_neg \\
\midrule
       activities &        4 &     8 &        279 &      0.34 &        262 &      0.13 \\
          complex &        2 &     3 &        154 &      0.25 &        123 &      0.07 \\
           gender &        1 &    10 &        122 &      0.97 &        150 &      0.46 \\
    part-material &        1 &     2 &         50 &      0.64 &         43 &      0.14 \\
            parts &        2 &     9 &        154 &      0.69 &        111 &      0.17 \\
       perceptual &        2 &     8 &        109 &      0.72 &        128 &      0.27 \\
 perceptual-color &        5 &     8 &        129 &      0.60 &        379 &      0.09 \\
 perceptual-shape &        2 &     3 &         73 &      0.40 &         39 &      0.15 \\
   percetual-heat &        3 &     5 &        140 &      0.54 &        101 &      0.12 \\
\bottomrule
\end{tabular}



## Relations

In [11]:
# def get hypotheses

from analyze_evidence import get_properties, get_relation_overview
import pandas as pd

In [37]:
props = get_properties()
#props = [p for p in props if p != 'female']
model = 'wiki'
relation_table = get_relation_overview(props, model, rel_type = 'top')
df = pd.DataFrame(relation_table)
df = df.sort_values('p_evidence', ascending = False)
print(df.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
            relation &  total\_concepts &  p\_evidence \\
\midrule
          gender-all &             152 &        0.78 \\
 typical\_of\_property &             127 &        0.68 \\
  affording\_activity &             407 &        0.59 \\
  typical\_of\_concept &             572 &        0.57 \\
    implied\_category &             629 &        0.50 \\
      afforded\_usual &             172 &        0.41 \\
          gender-few &             208 &        0.33 \\
    afforded\_unusual &              73 &        0.19 \\
 variability\_limited &             562 &        0.17 \\
             unusual &             578 &        0.16 \\
                rare &             308 &        0.15 \\
            creative &             152 &        0.12 \\
          impossible &             554 &        0.11 \\
    variability\_open &             443 &        0.08 \\
\bottomrule
\end{tabular}



In [33]:
props = get_properties()
model = 'wiki'
relation_table = get_relation_overview(props, model, rel_type = 'hyp_top')
df = pd.DataFrame(relation_table)
df = df.sort_values('p_evidence', ascending = False)
print(df.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
            relation &  total\_concepts &  p\_evidence \\
\midrule
          gender-all &             152 &        0.78 \\
 typical\_of\_property &             211 &        0.59 \\
  affording\_activity &             555 &        0.54 \\
  typical\_of\_concept &              19 &        0.53 \\
      afforded\_usual &             211 &        0.37 \\
          gender-few &             208 &        0.33 \\
    implied\_category &              45 &        0.27 \\
    afforded\_unusual &              50 &        0.24 \\
 variability\_limited &             803 &        0.20 \\
                rare &             244 &        0.15 \\
             unusual &             477 &        0.14 \\
            creative &             140 &        0.13 \\
          impossible &             553 &        0.11 \\
    variability\_open &             285 &        0.04 \\
\bottomrule
\end{tabular}



In [39]:
props = get_properties()
#props = [p for p in props if p != 'female']
model = 'giga_full'
relation_table = get_relation_overview(props, model, rel_type = 'top')
df = pd.DataFrame(relation_table)
df = df.sort_values('p_evidence', ascending = False)
print(df.to_latex(index=False))


\begin{tabular}{lrr}
\toprule
            relation &  total\_concepts &  p\_evidence \\
\midrule
          gender-all &             152 &        0.68 \\
 typical\_of\_property &             127 &        0.57 \\
  affording\_activity &             407 &        0.53 \\
  typical\_of\_concept &             572 &        0.52 \\
    implied\_category &             629 &        0.41 \\
      afforded\_usual &             172 &        0.38 \\
          gender-few &             208 &        0.23 \\
 variability\_limited &             562 &        0.19 \\
            creative &             152 &        0.17 \\
             unusual &             578 &        0.16 \\
                rare &             308 &        0.15 \\
    afforded\_unusual &              73 &        0.15 \\
          impossible &             554 &        0.10 \\
    variability\_open &             443 &        0.07 \\
\bottomrule
\end{tabular}



In [34]:
props = get_properties()
model = 'giga_full'
relation_table = get_relation_overview(props, model, rel_type = 'hyp_top')
df = pd.DataFrame(relation_table)
df = df.sort_values('p_evidence', ascending = False)
print(df.to_latex(index=False))

\begin{tabular}{lrr}
\toprule
            relation &  total\_concepts &  p\_evidence \\
\midrule
          gender-all &             152 &        0.68 \\
 typical\_of\_property &             211 &        0.52 \\
  affording\_activity &             555 &        0.49 \\
  typical\_of\_concept &              19 &        0.42 \\
      afforded\_usual &             211 &        0.37 \\
          gender-few &             208 &        0.23 \\
 variability\_limited &             803 &        0.21 \\
    implied\_category &              45 &        0.18 \\
            creative &             140 &        0.18 \\
    afforded\_unusual &              50 &        0.18 \\
             unusual &             477 &        0.14 \\
                rare &             244 &        0.12 \\
          impossible &             553 &        0.10 \\
    variability\_open &             285 &        0.03 \\
\bottomrule
\end{tabular}



## Development

In [216]:
s1 = {1,2,3}
s2 = {2,3,1}
s1 ==s2

True

In [214]:
prop = 'yellow'


In [19]:



properties = ['fly']
prop_cat_syn_dict = collect_search_log(properties)
prop_cat_syn_dict

defaultdict(list,
            {'swim': [{'bird': {"Synset('bird.n.01')", "Synset('bird.n.02')"}},
              {'fish': {"Synset('fish.n.01')", "Synset('fish.n.02')"}},
              {'mammal': {"Synset('mammal.n.01')"}},
              {'bird': {"Synset('bird.n.01')", "Synset('bird.n.02')"}},
              {'fish': {"Synset('fish.n.01')", "Synset('fish.n.02')"}},
              {'mammal': {"Synset('mammal.n.01')"}}]})